# Threat Hunting
### Hunting `PowerShell Execution` 

```
@Author: SK
```

### Hypothesis
Adversaries leverage `PowerShell to execute code` within the environmen

### Tradecraft
Using PowerShell to perform a number of actions, including, discovery of information & code execution. Therefore hunting/analysis of powershell artifacts left when the powershell  is used in the environment.

### Initialization

In [1]:
## Import Spark Session  
from pyspark.sql import SparkSession

#### Creating a spark session 

In [2]:
# Creatig a spark session
spark = SparkSession.builder.master('local[*]').appName('Testing').getOrCreate()

# Setting the spark log level to ERROR
spark.sparkContext.setLogLevel("ERROR")

# Set spark configuration to case sensitive
spark.conf.set('spark.sql.caseSensitive', 'true')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 07:34:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Loading Data

In [3]:
# Set the output fields to 1000
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# read the json file and stora it into a dataframe named dataset
empire = spark.read.json('empire_launcher_vbs_2020-09-04160940.json')

# Createa a table named empire for the dataset
empire.createOrReplaceTempView("empire")

## Knowing the DataSet

In [4]:
# Show the dataset columns (Identify the fiels we're looking for)
empire.columns

['@timestamp',
 '@version',
 'AccessList',
 'AccessMask',
 'AccessReason',
 'AccountName',
 'AccountType',
 'ActivityID',
 'Application',
 'Archived',
 'AuthenticationPackageName',
 'CallTrace',
 'Category',
 'Channel',
 'CommandLine',
 'Company',
 'ContextInfo',
 'CreationUtcTime',
 'CurrentDirectory',
 'Description',
 'DestAddress',
 'DestPort',
 'DestinationHostname',
 'DestinationIp',
 'DestinationIsIpv6',
 'DestinationPort',
 'DestinationPortName',
 'Details',
 'Device',
 'Direction',
 'DisabledPrivilegeList',
 'Domain',
 'ERROR_EVT_UNRESOLVED',
 'ElevatedToken',
 'EnabledPrivilegeList',
 'EventCountTotal',
 'EventID',
 'EventIdx',
 'EventReceivedTime',
 'EventTime',
 'EventType',
 'EventTypeOrignal',
 'ExecutionProcessID',
 'FileVersion',
 'FilterRTID',
 'GrantedAccess',
 'GroupMembership',
 'HandleId',
 'Hashes',
 'Hostname',
 'Image',
 'ImageLoaded',
 'ImpersonationLevel',
 'Initiated',
 'IntegrityLevel',
 'IpAddress',
 'IpPort',
 'IsExecutable',
 'KeyLength',
 'Keywords',
 'La

---
#### Hunting

#### Analytics 1: Within the classic powershell log, event ID 400 indicates when a new PowerShell host process has started. To filter use `powershell.exe` or capture every single powershell hosts. 

In [5]:
# Select @timestamp, Channel, Hostname were Channel == 'Windows PowerShell' or 'Microsoft-Windows-PowerShell/Operational' AND EventID == 400 or 4103 from the dataset
query = '''
        SELECT '@timestamp', Channel, Hostname 
        FROM empire 
        WHERE (Channel = 'Windows PowerShell' OR Channel = 'Microsoft-Windows-PowerShell/Operational') 
        AND (EventID = 400 OR EventID = 4103)
        '''

dataset = spark.sql(query)

dataset.show(10, False)

+----------+----------------------------------------+---------------------------+
|@timestamp|Channel                                 |Hostname                   |
+----------+----------------------------------------+---------------------------+
|@timestamp|Windows PowerShell                      |WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Microsoft-Windows-PowerShell/Operational|WORKSTATION5.theshire.local|
|@timestamp|Micr

#### Analytics 2: Look for non-interactive PowerShell session might be a sign of a PowerShell being executed by another application in the background

In [10]:
# Select Hostname, Channel, EventID, NewProcessName, ParentProcessName where lower(Channel) == 'security' and EventID == 4688 and 'PowerShell' in NewProcessName and ParentProcessName != 'explorer' from the dataset
query = '''
		SELECT Hostname, Channel, EventID, NewProcessName, ParentProcessName
		FROM empire
		WHERE lower(Channel) = 'security' 
		AND EventID = 4688 
		AND NewProcessName LIKE '%PowerShell%' 
		AND NOT ParentProcessName LIKE '%explorer'
		'''

dataset = spark.sql(query)

dataset.show(10, False)

+---------------------------+--------+-------+---------------------------------------------------------+-------------------------------+
|Hostname                   |Channel |EventID|NewProcessName                                           |ParentProcessName              |
+---------------------------+--------+-------+---------------------------------------------------------+-------------------------------+
|WORKSTATION5.theshire.local|Security|4688   |C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|C:\Windows\System32\wscript.exe|
+---------------------------+--------+-------+---------------------------------------------------------+-------------------------------+



### Analytics 3: Monitor for process loading PowerShell DLL `system.management.automation`

In [18]:
# Monitor for process loading PowerShell DLL `system.management.automation` from the dataset
query = '''
        SELECT '@timestamp', Hostname, Image, ImageLoaded
        FROM empire
        WHERE Channel = 'Microsoft-Windows-Sysmon/Operational' 
        AND EventID = 7 
        AND (lower(Description) = 'system.management.automation' OR lower(ImageLoaded) LIKE '%system.management.automation%')
        '''

dataset = spark.sql(query)

dataset.show(10, False)

+----------+---------------------------+---------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|@timestamp|Hostname                   |Image                                                    |ImageLoaded                                                                                                                             |
+----------+---------------------------+---------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|@timestamp|WORKSTATION5.theshire.local|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|C:\Windows\assembly\NativeImages_v4.0.30319_64\System.Manaa57fc8cc#\8c159a697bcf84de413ace36e043a6c7\System.Management.Automation.ni.dll|
+----------+---------------------------+----------------

### Analytics 5: Monitor the PSHost* pipes to find another powerhell execution

In [19]:
query = '''
        SELECT '@timestamp', Hostname, Image, PipeName
        FROM empire
        WHERE Channel = 'Microsoft-Windows-Sysmon/Operational' 
        AND EventID = 17 
        AND lower(PipeName) LIKE '\\pshos%'
        '''

dataset = spark.sql(query)

dataset.show(10, False)

+----------+--------+-----+--------+
|@timestamp|Hostname|Image|PipeName|
+----------+--------+-----+--------+
+----------+--------+-----+--------+



### Analytics 6: looking for the `"PowerShell Name Pip IPC"` event indicate the name of the process that the AppDomain PoweShell process executed.

In [21]:
# Select @timestamp, Hostname, Message where EventID = 53504 for Channel 'Microsodt-Windows-PowerShell/Operational' from the dataset
query = '''
        SELECT '@timestamp', Hostname, Message
        FROM empire
        WHERE Channel = 'Microsoft-Windows-PowerShell/Operational' 
        AND EventID = 53504
        '''

dataset = spark.sql(query)

dataset.show(10, False)

+----------+---------------------------+-------------------------------------------------------------------------------------------------------+
|@timestamp|Hostname                   |Message                                                                                                |
+----------+---------------------------+-------------------------------------------------------------------------------------------------------+
|@timestamp|WORKSTATION5.theshire.local|Windows PowerShell has started an IPC listening thread on process: 2316 in AppDomain: DefaultAppDomain.|
+----------+---------------------------+-------------------------------------------------------------------------------------------------------+

